#Import


In [0]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.nn.functional as F
import torchvision.utils as vutils
from PIL import ImageFile
from zipfile import ZipFile
import os

#Global Variables

In [0]:
batchSize = 1
img_size = 128
n_channel = 3 #Grey
n_encode = 64

n_disc = 16 #discriminator base output
n_gen = 64 # generator base output 

n_z = 50 #Latent vector size              
n_l = 10 #label number 
n_age = int(n_z/n_l) #5 ??? age category
n_gender = int(n_z/2) #25 ??? gender ?????

use_cuda = torch.cuda.is_available()
print(use_cuda)

des_dir="./police_db_new/police_db_new"

True


In [0]:
!rm -r "./police_db_new"

#Connect to drive

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


#Extract data


In [0]:
file_name = "./drive/My Drive/Research Project/police_db_newcleaned.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("Unzipped file")
  
file_name2 = "./drive/My Drive/Research Project/Training/train_11.zip"

with ZipFile(file_name2, 'r') as zip:
  zip.extractall()
  print("Unzipped file")

#file_name3 = "./drive/My Drive/Research Project/Training/train_12.zip"

#with ZipFile(file_name3, 'r') as zip:
 # zip.extractall()
 # print("Unzipped file")

#file_name4 = "./drive/My Drive/Research Project/Training/train_13.zip"

#with ZipFile(file_name4, 'r') as zip:
 # zip.extractall()
  #print("Unzipped file")


Unzipped file
Unzipped file


#Create op directory

In [0]:
#folders=['/A00_5F','/A00_5M','/A05_10F','/A05_10M','/A10_16F','/A10_16M','/A16_24F','/A16_24M','/A24_32F','/A24_32M','/A32_42F','/A32_42M','/A42_52F','/A42_52M','/A52_62F','/A52_62M','/A62_70F','/A62_70M','/A70_120F','/A70_120M']

outf='./aged_pr_hackathon'
#outf10='./output12'
#outf13='./output13'

#if(os.path.exists(outf11) and os.path.exists(outf13)) :
if(os.path.exists(outf)) :
  print("File exists")  
else:
  os.mkdir(outf)
  #os.mkdir(outf10)
  #os.mkdir(outf13)
  print("File created.")

for j in folders:
 if(os.path.exists(outf+j)):
  print("File exists")  
 else:
  os.mkdir(outf+j)
  print("File created.")


"""
file_name2 = "./drive/My Drive/Research Project/Training/train_11.zip"

with ZipFile(file_name2, 'r') as zip:
  zip.extractall()
  print("Unzipped file")
"""

File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.
File created.


'\nfile_name2 = "./drive/My Drive/Research Project/Training/train_11.zip"\n\nwith ZipFile(file_name2, \'r\') as zip:\n  zip.extractall()\n  print("Unzipped file")\n'

In [0]:
#!rm -rf ./Examples

#Dataloader

In [0]:
#load images
#!ls "/content/drive/My Drive/Research Project"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([transforms.Resize(128),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5,0.5),(0.5,0.5,0.5)),
                           ]))
#for i, j in dataset:
 # print(i,j) 
#dataset contains i=image tensor values, j = folder number 

#divide all images into batches of 20
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=False, drop_last=True)
#torch.save(dataloader, "%s/dataloader.pth"%(outf))

#dataloader=torch.load("./train_11/dataloader.pth")

for i,(j,k) in enumerate(dataloader):
      print(i,k)

#images/ batch size 20 =number of batches
#i=batch number, j=list of 20 tensor values, k=list of folder numbers


(0, tensor([0]))
(1, tensor([0]))
(2, tensor([0]))
(3, tensor([0]))
(4, tensor([1]))
(5, tensor([1]))
(6, tensor([1]))
(7, tensor([1]))
(8, tensor([1]))
(9, tensor([1]))
(10, tensor([1]))
(11, tensor([2]))
(12, tensor([2]))
(13, tensor([2]))
(14, tensor([2]))
(15, tensor([2]))
(16, tensor([2]))
(17, tensor([2]))
(18, tensor([3]))
(19, tensor([3]))
(20, tensor([3]))
(21, tensor([3]))
(22, tensor([3]))
(23, tensor([3]))
(24, tensor([3]))
(25, tensor([3]))
(26, tensor([3]))
(27, tensor([3]))
(28, tensor([4]))
(29, tensor([4]))
(30, tensor([4]))
(31, tensor([4]))
(32, tensor([4]))
(33, tensor([4]))
(34, tensor([4]))
(35, tensor([4]))
(36, tensor([4]))
(37, tensor([4]))
(38, tensor([4]))
(39, tensor([4]))
(40, tensor([4]))
(41, tensor([4]))
(42, tensor([4]))
(43, tensor([4]))
(44, tensor([4]))
(45, tensor([4]))
(46, tensor([4]))
(47, tensor([4]))
(48, tensor([4]))
(49, tensor([4]))
(50, tensor([4]))
(51, tensor([4]))
(52, tensor([4]))
(53, tensor([4]))
(54, tensor([4]))
(55, tensor([4]))
(5

#Layers
Encoder (i) img data- conv2D, relu x4 (ii) Linear of (i)

Generator (i) Concatenate z,age,gender- linear, relu (ii) Conv2d transpose, relu x3 tanh of (i)

Dimg (i) img - conv2D (ii) concatenate(age,gender) - convtranspose + relu (iii) concatenate (i) and (ii) - conv2d + relu x3 (iv) Linear, relu of (iii) (v) Linear, sigmoid of (iv) (vi) Linear, softmax of (iv)

Dz linear + relu x3, sigmoid of z

#Encoder

In [0]:
class Encoder(nn.Module): ##defining Encoder as a subclass of nn.Module
    def __init__(self): #runs initially #self=any arguments passed
        super(Encoder,self).__init__() #define super class
        self.layer1 = nn.Sequential( #feed-forward fully connected layers
             nn.Conv2d(n_channel,n_encode,5,2,2), #(input,output) where output=2Dconv(input) 
            nn.ReLU(),
            
            nn.Conv2d(n_encode,2*n_encode,5,2,2), #prev output becomes input
            nn.ReLU(),
            
            nn.Conv2d(2*n_encode,4*n_encode,5,2,2),
            nn.ReLU(),
            
            nn.Conv2d(4*n_encode,8*n_encode,5,2,2),
            nn.ReLU(),
        
        )
        self.layer2 = nn.Linear(8*n_encode*8*8,50)

        
    def forward(self,x):
        conv = self.layer1(x).view(-1,8*n_encode*8*8)
        out = self.layer2(conv)
        return out

#Generator

In [0]:
class Generator(nn.Module): #(z, age_ohe, img_gender_v)
    def __init__(self):
        super(Generator,self).__init__()
        self.fc = nn.Sequential(nn.Linear(n_z+n_l*n_age+n_gender,#Creating a layer w linear transformation of args
                                          8*8*n_gen*16),# This FC layer collects all the features collected by breaking down inp image by previous multiple convolutions
                                  #together into an object that you want the network to recognize
                                nn.ReLU()) 
        
        self.upconv= nn.Sequential( #upsampling - transforms a small input to a large output
            nn.ConvTranspose2d(16*n_gen,8*n_gen,4,2,1),
            nn.ReLU(), #ConvTranspose upsamples with trainable kernels
            
            nn.ConvTranspose2d(8*n_gen,4*n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(4*n_gen,2*n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(2*n_gen,n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(n_gen,n_channel,3,1,1),
            nn.Tanh(),
        
        )
        
    def forward(self,z,age,gender):
        l = age.repeat(1,n_age) # (cols, rows) 1 x n_age matrix with age as every element 
        k = gender.view(-1,1).repeat(1,n_gender) # transpose gender and repeat
        z=z.float() # converting to float
        l=l.float()
        k=k.float()
       # print(dim(z),dim(l),dim(k))
        x = torch.cat([z,l,k],dim=1) #concatenate
        fc = self.fc(x).view(-1,16*n_gen,8,8) 
        out = self.upconv(fc)
        return out

#Fixed_l

In [0]:
#doesnt need to be saved for [32,64] run because it is standard
fixed_l = -torch.ones(10*10).view(10,10) #initialize 80x10 matrix with value -1
for i,l in enumerate(fixed_l): #fix 1 for each of the 80 tensors
    l[i//1] = 1
fixed_l_v = Variable(fixed_l)
if use_cuda:
    fixed_l_v = fixed_l_v.cuda() 

#Initialise nets

In [0]:
if use_cuda:
    netE = Encoder().cuda()
    netG = Generator().cuda()
else:
    netE = Encoder()
    netG = Generator()

#Initialise weights

In [0]:
def weights_init(m):  #weights calculated according to class name    
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find("Linear") !=-1: #if conv or linear
        m.weight.data.normal_(0.0, 0.02) #mean, standard deviation
    elif classname.find('BatchNorm') != -1: #batch normalization
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


netE.apply(weights_init)
netG.apply(weights_init) 

Generator(
  (fc): Sequential(
    (0): Linear(in_features=125, out_features=65536, bias=True)
    (1): ReLU()
  )
  (upconv): Sequential(
    (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): ConvTranspose2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Tanh()
  )
)

#Loading a model 

In [0]:
"""
#Encoder
checkpoint = torch.load("./train_11/netE.pth")
netE.load_state_dict(checkpoint)
netE.eval()

#Generator
checkpoint1 = torch.load("./train_11/netG.pth")
netG.load_state_dict(checkpoint1)
netG.eval()
"""

'\n#Encoder\ncheckpoint = torch.load("./train_11/netE.pth")\nnetE.load_state_dict(checkpoint)\nnetE.eval()\n\n#Generator\ncheckpoint1 = torch.load("./train_11/netG.pth")\nnetG.load_state_dict(checkpoint1)\nnetG.eval()\n'

#Optimiser

In [0]:

#implementing Adam optimization algorithm 
optimizerE = optim.Adam(netE.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))
#lr= learning rate
#betas= coefficients for averages


#Main

In [0]:
num_list=[0]
allImgNames=[]
allFolderNames = os.listdir(des_dir)
print(allFolderNames)
allFolderNames.sort()
for i,folderNames in enumerate(allFolderNames):
  FolderImgNames=os.listdir(des_dir+'/'+folderNames)
  FolderImgNames.sort()
  print(FolderImgNames)
  allImgNames.append(FolderImgNames)
  #allImgNames.sort()
  num=len(os.listdir(des_dir+'/'+folderNames))
  prev=num_list[i]
#  print(prev)
  num_list.append(num+prev)
print(num_list)
print(allImgNames)

['A70_120M', 'A24_32M', 'A42_52M', 'A10_16F', 'A16_24F', 'A24_32F', 'A52_62M', 'A32_42M', 'A42_52F', 'A00_5F', 'A52_62F', 'A05_10F', 'A62_70M', 'A70_120F', 'A05_10M', 'A10_16M', 'A16_24M', 'A62_70F', 'A32_42F', 'A00_5M']
['351 001 - #_a2e79689-b3e5-4d2b-af2a-38f278c9232e2202.jpg', 'AVADAVALLI#_0d129288-6023-4934-9bc9-2d5d7089f1ca1382.jpg', 'Cr nO 248 #_500ea238-7bf4-4aeb-bf59-919640f72c6f385.jpg', 'papu#_1f25489e-ac24-478f-a95f-900c5ad2624e2123.jpg']
['178#_e1b3fe7f-b0a7-419d-9435-0bd2ab2dde7e373.jpg', '2388 - Cop#_63e20ee8-3372-4b2b-9973-8b53ed32c3261402.jpg', '377-18, PH#_96dba0a8-28ab-4a8d-9e7f-e785ee60c9631378.jpeg', 'Cr No 134-#_1cd06241-56f5-472b-8b6a-76cb721e67bb1540.jpg', 'Cr No 134-#_adc73e7a-ed60-4509-8313-cf60e74e8b1c1540.jpg', 'MOHAN RAJ#_d7782620-4275-4229-9931-1dc48765667a1382.jpg', 'SAMANITHA#_11942597-b589-49d3-8113-57530acfbb9915.jpg']
['137 - 4 Si#_a9e5f1e4-6395-4f2c-b3bc-099e8d0ec73f1250.jpg', '248-18 Ira#_f5fc2902-3851-4699-a63b-4ce20f70da991460.jpg', 'CRNO 219-2#_9

In [0]:
#for i in range(20): # random examples to demonstrate
for batch_num, (img_data,folder_list) in enumerate(dataloader):
  img_gender = (folder_list%2)*2-1
  #convert to variable
  img_data_v = Variable(img_data) 
  img_age_v = Variable((folder_list-(folder_list%2))*4 ).view(-1,1) #+transpose
  img_gender_v = Variable(img_gender.float())
          
  if use_cuda: #appending device=cuda to variable
        img_data_v = img_data_v.cuda()
        img_age_v = img_age_v.cuda()
        img_gender_v = img_gender_v.cuda()  
      
  #initialize noise for data and gender by taking first 8 values in first loop and repeating
  fixed_img_v = Variable(img_data[:1].repeat(10,1,1,1))
  fixed_g_v = Variable(img_gender[:1].view(-1,1).repeat(10,1))
              
  if use_cuda:
        fixed_img_v = fixed_img_v.cuda()
        fixed_g_v = fixed_g_v.cuda()  

  #Encoder
  checkpoint = torch.load("./train_11/netE.pth")
  netE.load_state_dict(checkpoint)
  netE.eval()

#Generator
  checkpoint1 = torch.load("./train_11/netG.pth")
  netG.load_state_dict(checkpoint1)
  netG.eval()
                         

  checkpoint4 = torch.load("./train_11/optimizerE.pth")
  optimizerE.load_state_dict(checkpoint4)

  checkpoint7 = torch.load("./train_11/optimizerG.pth")
  optimizerG.load_state_dict(checkpoint7)
   
  #for every image     
  fixed_z = netE(fixed_img_v)
  #print(fixed_z)
  #vutils.save_image(fixed_z.data,
  #                '%s/reconst_op%03d.png' %(outf,i),
  #               normalize=True)
 # print()
  fixed_fake = netG(fixed_z,fixed_l_v,fixed_g_v)
  #print(fixed_fake)
  offset=num_list[folder_list.item()]
  img_name=allImgNames[folder_list.item()][batch_num-offset]
  print(batch_num, folder_list, img_name)
 # print(folders[idx*2])
 
  for idx in range(10): #0 1 2 3 4 5 6 7 8 9
   if(img_gender==-1): #folders 0 2 4 6 8
     vutils.save_image(fixed_fake.data[idx],
                  '%s%s/%s.jpg' % (outf,folders[idx*2],img_name),
                 normalize=True) #save img
   else: #folders 1 3 5
     vutils.save_image(fixed_fake.data[idx],
                  '%s%s/%s.jpg' % (outf,folders[idx*2+1],img_name),
                 normalize=True) #save img
                 
  vutils.save_image(fixed_fake.data,
                  '%s/%s.jpg' % (outf,img_name),
                  normalize=True) #save img 


In [0]:
#!rm -rf ./output

In [0]:
!zip -r ./aged_pr_hackathon.zip ./aged_pr_hackathon
!cp "aged_pr_hackathon.zip" "/content/drive/My Drive/Research Project/aged_pr_hackathon.zip"